# Redis

>[Redis（リモートディクショナリサーバー）](https://en.wikipedia.org/wiki/Redis)は、インメモリデータ構造ストアであり、分散インメモリキーバリューデータベース、キャッシュ、およびメッセージブローカーとして使用され、オプションで耐久性があります。

このノートブックでは、[Redisベクターデータベース](https://redis.com/solutions/use-cases/vector-database/)に関連する機能の使い方を示します。

## インストール方法

In [1]:
!pip install redis

`OpenAIEmbeddings`を使用するためには、OpenAI APIキーを取得する必要があります。

In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

## 例

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.redis import Redis

In [3]:
from langchain.document_loaders import TextLoader

loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [4]:
rds = Redis.from_documents(docs, embeddings, redis_url="redis://localhost:6379",  index_name='link')

ConnectionError: Error 10061 connecting to localhost:6379. 対象のコンピューターによって拒否されたため、接続できませんでした。.

In [ ]:
rds.index_name

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
results = rds.similarity_search(query)
print(results[0].page_content)

In [5]:
print(rds.add_texts(["Ankush went to Princeton"]))

NameError: name 'rds' is not defined

In [ ]:
query = "Princeton"
results = rds.similarity_search(query)
print(results[0].page_content)

In [ ]:
# 既存のインデックスから読み込む
rds = Redis.from_existing_index(embeddings, redis_url="redis://localhost:6379", index_name='link')

query = "What did the president say about Ketanji Brown Jackson"
results = rds.similarity_search(query)
print(results[0].page_content)

## Redisを使った検索

ここでは、ベクトルストアを検索に使用するためのさまざまなオプションについて説明します。

検索には3つの異なる方法があります。デフォルトでは、意味的類似性を使用します。

In [ ]:
retriever = rds.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents(query)

また、`similarity_limit` を検索方法として使用することもできます。これは、ドキュメントが十分に類似している場合にのみドキュメントを返します。

In [ ]:
retriever = rds.as_retriever(search_type="similarity_limit")

In [ ]:
# ここでは、関連するドキュメントがないため結果が返されないことがわかります。
retriever.get_relevant_documents("where did ankush go to college?")